In [ ]:
import json
import os
from openai import OpenAI
import csv
from io import StringIO
import pandas as pd
#!pip install matplotlib
import matplotlib
!pip install tiktoken
import tiktoken

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"), organization = os.getenv("OPENAI_ORGANIZATION"))

In [2]:
# Set System Role

SystemRole = """You are an assistant helping to categorize responses from parents that have decided to not vaccinate their children.  
The survey that was given to these parents had pre-defined categories for why they have decided to not vaccinate their children.  
The following are the categories: 
[1] Vaccination site too far 
[2] Vaccination schedule not known 
[3] mother too busy 
[4] Family problems, including maternal illness 
[5] Sick child, not sent
[6] Sick child, sent but not vaccinated
[7] Long wait
[8] Rumors
[9] Don't believe in vaccination
[10] Fear of side effects
[11] Site and/or time of vaccination not known
[12] Ignore the need for vaccination
[13] Ignores need to return for 2nd or 3rd dose
[14] Bad ideas about contraindications
[15] Inappropriate timing of vaccination
[16] Absent vaccinator
[17] Vaccine not available
[18] Vaccination session canceled
[19] High cost of vaccination or SMC session
[20] Sick child, brought in but did not receive vaccination
[21] Religious censorship
[22] Negative attitude of the spouse, father or guardian of the child towards vaccination
[23] Provider on strike
[24] COVID-19 lockdown
[25] Travel or displacement
[26] War, armed conflict, ethnic conflict
[27] Fear of COVID vaccine
[28] Don't know the reason
[29] Vaccine not mentioned on card
[30] Other
[31] COVID-19 (fear of catching, or vague)
[32] Child too young to have completed schedule
[33] Vague or unclear response
[34] Working in the field/agricultural work
[35] Claims to have received all vaccines afterall
[36] Vaccine-related items not available (syringe, cards)
[37] Forgot
[38] Insufficient number of children at session 
[39] 'Negligence' 
[40] Born in a different place/home, possible inability to vaccinate related to that
[41] Missed birth dose
[42] Issues with health workers (negatve experience, no reminder, and distrust)
[43] No card to remind of appointment or dates not on card

Your role as the assistant is to assess the unstructured responses from the survey of parents.
If a reason in the response likely corresponds to one of the categories, provide only the category number 
If a response does not fit into other categories and a new category needs to be created, provide the number 30 
Only output one category number matching the most relevant category.  
    
Here are two examples:

user:  We don't live in this country. 
assistant:  30
    
user:  I could not find the time.
assistant: 3
"""

In [3]:
# Set System Role

SystemRoleCOT = """You are an assistant helping to categorize responses from parents that have decided to not vaccinate their children.  
The survey that was given to these parents had pre-defined categories for why they have decided to not vaccinate their children.  
The following are the categories: 
[1] Vaccination site too far 
[2] Vaccination schedule not known 
[3] mother too busy 
[4] Family problems, including maternal illness 
[5] Sick child, not sent
[6] Sick child, sent but not vaccinated
[7] Long wait
[8] Rumors
[9] Don't believe in vaccination
[10] Fear of side effects
[11] Site and/or time of vaccination not known
[12] Ignore the need for vaccination
[13] Ignores need to return for 2nd or 3rd dose
[14] Bad ideas about contraindications
[15] Inappropriate timing of vaccination
[16] Absent vaccinator
[17] Vaccine not available
[18] Vaccination session canceled
[19] High cost of vaccination or SMC session
[20] Sick child, brought in but did not receive vaccination
[21] Religious censorship
[22] Negative attitude of the spouse, father or guardian of the child towards vaccination
[23] Provider on strike
[24] COVID-19 lockdown
[25] Travel or displacement
[26] War, armed conflict, ethnic conflict
[27] Fear of COVID vaccine
[28] Don't know the reason
[29] Vaccine not mentioned on card
[30] Other
[31] COVID-19 (fear of catching, or vague)
[32] Child too young to have completed schedule
[33] Vague or unclear response
[34] Working in the field/agricultural work
[35] Claims to have received all vaccines afterall
[36] Vaccine-related items not available (syringe, cards)
[37] Forgot
[38] Insufficient number of children at session 
[39] 'Negligence' 
[40] Born in a different place/home, possible inability to vaccinate related to that
[41] Missed birth dose
[42] Issues with health workers (negatve experience, no reminder, and distrust)
[43] No card to remind of appointment or dates not on card

Your role as the assistant is to assess the unstructured responses from the survey of parents.
If a reason in the response likely corresponds to one of the categories, provide only the category number 
If a response does not fit into other categories and a new category needs to be created, provide the number 30 
Only output one category number matching the most relevant category.  
Explain your reasoning step by step for placing the response in the category.
Output the category number in the first line and the reasoning in the subsequent line.
    
Here are two examples:

user:  We don't live in this country. 
assistant:  30\n
The reason is that the family does not live in the country.  There is not a category describing the family being from somwhere else.
    
user:  I could not find the time.
assistant: 3\n
The reason is that the mother is too busy and could not find the time to vaccinate the children.  
"""

In [4]:
SystemRoleM = """You are an assistant helping to categorize responses from parents that have decided to not vaccinate their children.  
The survey that was given to these parents had pre-defined categories for why they have decided to not vaccinate their children.  
The following are the categories: 
[1] Vaccination site too far 
[2] Vaccination schedule not known 
[3] mother too busy 
[4] Family problems, including maternal illness 
[5] Sick child, not sent
[6] Sick child, sent but not vaccinated
[7] Long wait
[8] Rumors
[9] Don't believe in vaccination
[10] Fear of side effects
[11] Site and/or time of vaccination not known
[12] Ignore the need for vaccination
[13] Ignores need to return for 2nd or 3rd dose
[14] Bad ideas about contraindications
[15] Inappropriate timing of vaccination
[16] Absent vaccinator
[17] Vaccine not available
[18] Vaccination session canceled
[19] High cost of vaccination or SMC session
[20] Sick child, brought in but did not receive vaccination
[21] Religious censorship
[22] Negative attitude of the spouse, father or guardian of the child towards vaccination
[23] Provider on strike
[24] COVID-19 lockdown
[25] Travel or displacement
[26] War, armed conflict, ethnic conflict
[27] Fear of COVID vaccine
[28] Don't know the reason
[29] Vaccine not mentioned on card
[30] Other
[31] COVID-19 (fear of catching, or vague)
[32] Child too young to have completed schedule
[33] Vague or unclear response
[34] Working in the field/agricultural work
[35] Claims to have received all vaccines afterall
[36] Vaccine-related items not available (syringe, cards)
[37] Forgot
[38] Insufficient number of children at session 
[39] 'Negligence' 
[40] Born in a different place/home, possible inability to vaccinate related to that
[41] Missed birth dose
[42] Issues with health workers (negatve experience, no reminder, and distrust)
[43] No card to remind of appointment or dates not on card

Your role as the assistant is to assess the unstructured responses from the survey of parents.
If a reason in the response likely corresponds to one of the categories, provide only the category number 
If a response does not fit into other categories and a new category needs to be created, provide the number 30 
Only output one category number matching the most relevant category.
"""

In [ ]:
# Prepare Data set

numOfTrainingSamples = 799
numOfTestingSamples = 200

with open('ecv_reasons_novx 5.csv', 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()

# Assuming 'content' is your CSV file content as a string
content_io = StringIO(content)

# Create a CSV DictReader object
reader = csv.DictReader(content_io)

rows = list(reader)

# Experiment with different files trying to clean up the encoding issues with the csv file

with open('ecv_reasons_novx 5Clean.csv', 'r', encoding='ISO-8859-1', errors='ignore') as file:
    content = file.read()

# Assuming 'content' is your CSV file content as a string
content_io = StringIO(content)

# Create a CSV DictReader object
reader = csv.DictReader(content_io)

rowsJP = list(reader)

print(rows[1]['reason'])
print(rows[1]['roy_categorized'])

print(rowsJP[1]['reason'])
print(rowsJP[1]['roy_categorized'])

In [6]:
## Write the training and testing data to a jsonl file for fine-tuning

# Specify the file name
file_name = 'VR_TrainingData6.jsonl'

# Initialize an empty list or it could already contain some existing conversation sessions
conversations = []

# Number of new conversations to add
number_of_new_conversations = numOfTrainingSamples  # Change this as needed

with open(file_name, 'w') as file:
    # Loop to add new conversation sessions
    for i in range(number_of_new_conversations):
        # New conversation session with a set of messages
        new_conversation = {
            "messages": [
                {"role": "system", "content": SystemRole},
                {"role": "user", "content": rows[i]['reason']},
                {"role": "assistant", "content": rows[i]['roy_categorized']}
            ]
        }
        # Append the new conversation to the list
        file.write(json.dumps(new_conversation) + '\n')
        #conversations.append(new_conversation)


# Specify the file name
file_name = 'VR_TestingData6.jsonl'

# Initialize an empty list or it could already contain some existing conversation sessions
conversations = []

# Number of new conversations to add
number_of_new_conversations = numOfTrainingSamples  # Change this as needed

with open(file_name, 'w') as file:
    # Loop to add new conversation sessions
    for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
        # New conversation session with a set of messages
        new_conversation = {
            "messages": [
                {"role": "system", "content": SystemRole},
                {"role": "user", "content": rows[i]['reason']},
                {"role": "assistant", "content": rows[i]['roy_categorized']}
            ]
        }
        # Append the new conversation to the list
        file.write(json.dumps(new_conversation) + '\n')
        #conversations.append(new_conversation)

In [ ]:
# Upload DataSet to OpenAI

client.files.create(
  file=open("VR_TrainingData6.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
client.files.create(
  file=open("VR_TestingData6.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
#Create a fine-tuning job and specify model.  Need to put your file ids in the training_file and validation_file fields

#client.fine_tuning.jobs.create(
#  training_file   = "",
#  validation_file = "",
#  model="gpt-3.5-turbo"
#)


In [ ]:
# Retrieve the state of a fine-tune"

client.fine_tuning.jobs.retrieve("")

In [ ]:
# Cancel a job if needed.

#client.fine_tuning.jobs.cancel("")

In [ ]:
#Look for the results of the fine-tuning job and plot some fitting metrics.

response = client.files.content("")

# Decode the binary content to a string
decoded_content = response.content.decode('utf-8')

# Use StringIO to convert the decoded string to a file-like object
data = StringIO(decoded_content)
df = pd.read_csv(data)

df.to_csv('ResultsFTFinal.csv', index=False)

# Now df is a Pandas DataFrame containing your CSV data
print(df)

# Example plotting code
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df['step'], df['train_loss'], label='Train Loss')
plt.xlabel('step')
plt.ylabel('Value')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(df['step'], df['train_accuracy'], label='Train Accuracy')
plt.xlabel('step')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
# Use the fine-tuned model to generate a completion.

completion = client.chat.completions.create(
  model="",
  messages=[
    {"role": "system", "content": SystemRole},
    {"role": "user", "content": "Too busy"}
  ]
)   
print(completion.choices[0].message.content)  

In [ ]:
# Fine-tune a GPT4 model

client.fine_tuning.jobs.create(
  training_file   = "",
  validation_file = "",
  model="gpt-4-0613"
)

In [ ]:
# Retrieve the state of a fine-tune"

client.fine_tuning.jobs.retrieve("")

In [ ]:
# Cancel a job
#client.fine_tuning.jobs.cancel("")

In [57]:
# List up to 10 events from a fine-tuning job
#test = client.fine_tuning.jobs.list_events(fine_tuning_job_id="", limit=500)


In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
#client.models.delete("")

In [12]:
#Helper Function to find matching numbers in two strings.

def find_matching_numbers(str1, str2):
    # Convert the first string into a list of integers
    list1 = [int(num) for num in str1.replace('[', '').replace(']', '').split(',')]
    
    # Convert the second string into a list of integers
    list2 = [int(num) for num in str2.replace('[', '').replace(']', '').split(',')]
    
    # Find intersection of both lists to get matching numbers
    matching_numbers = list(set(list1) & set(list2))
    
    return matching_numbers

In [ ]:
# Test the fine-tune GPT3.5 model with the testing data.

FT_Category = []
Diff = []
#DiffJP = []
count = 0
newCount = 0
savePredictionFT = []
FTTokens = 0

encoding = tiktoken.get_encoding("cl100k_base")

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
    try:
        completion = client.chat.completions.create(
              model="",
              messages=[
              {"role": "system", "content": SystemRole},
              {"role": "user", "content": rows[i]['reason']}
              ]
        )

        FTTokens += len(encoding.encode(SystemRole + rows[i]['reason']))
        
        predicted_category = completion.choices[0].message.content
        savePredictionFT.append(int(predicted_category))
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        #matching_numbersJP = find_matching_numbers(predicted_category,rowsJP[i]['JP'])
        FT_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
FTTokens += 199
print(str(FTTokens))


In [ ]:
# Test the fine-tune GPT4.0 model with the testing data.

FT4_Category = []
Diff = []
#DiffJP = []
count = 0
newCount = 0
savePredictionFT4 = []
FT4Tokens = 0

encoding = tiktoken.get_encoding("cl100k_base")

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
    try:
        completion = client.chat.completions.create(
              model="",
              messages=[
              {"role": "system", "content": SystemRole},
              {"role": "user", "content": rows[i]['reason']}
              ]
        )

        FT4Tokens += len(encoding.encode(SystemRole + rows[i]['reason']))
        
        predicted_category = completion.choices[0].message.content
        savePredictionFT4.append(int(predicted_category))
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        #matching_numbersJP = find_matching_numbers(predicted_category,rowsJP[i]['JP'])
        FT4_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
FTTokens += 199
print(str(FTTokens))

In [6]:
encoding = tiktoken.get_encoding("cl100k_base")

In [ ]:
# What's it getting wrong and why?  

import csv

count = 0
incorrectRows = []
# Define the additional key and its default value
additional_key = 'FT:Prediction'
additional_value = 0

# Determine the headers for the CSV based on the first item in rows, assuming all rows have similar structure
headers = list(rows[numOfTrainingSamples].keys()) + [additional_key]

with open('Wrong.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()  # Write the CSV headers
    for i in range(numOfTrainingSamples, numOfTrainingSamples + numOfTestingSamples):
        if not Diff[count]:
            # Copy the row and add the additional key-value pair
            row_copy = rows[i].copy()
            row_copy[additional_key] = savePrediction[count]
            
            writer.writerow(row_copy)
            incorrectRows.append(int(rows[i]['roy_categorized']))
        count += 1


import matplotlib.pyplot as plt

# Extracting the first 200 'roy_categorized' entries and converting them to integers
data = [int(row['roy_categorized']) for row in rows[:800]]

# Creating a histogram
plt.figure(figsize=(10, 6))
plt.hist(data, bins=43, color='blue', edgecolor='black')  # Adjust the number of bins as needed
plt.title('Histogram of Roy Training Data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.xticks(range(0, 45))
plt.show()

# Extracting the first 200 'roy_categorized' entries and converting them to integers
data = [int(row['roy_categorized']) for row in rows[800:999]]

# Creating a histogram
plt.figure(figsize=(10, 6))
plt.hist(data, bins=43, color='blue', edgecolor='black')  # Adjust the number of bins as needed
plt.title('Histogram of Roy Test Data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.xticks(range(0, 45))
plt.show()

# Extracting the first 200 'roy_categorized' entries and converting them to integers

# Creating a histogram
plt.figure(figsize=(10, 6))
plt.hist(incorrectRows, bins=43, color='blue', edgecolor='black')  # Adjust the number of bins as needed
plt.title('Histogram of Roy Test Data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.xticks(range(0, 45))
plt.show()

In [ ]:
# Test the gpt4 turbo on the validation set with the System Role.

FT_Category = []
Diff = []
count = 0
newCount = 0
savePredictionGPT4 = []
GPT4Tokens = 0

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
    try:
        completion = client.chat.completions.create(
              model="gpt-4-turbo-preview",
              messages=[
              {"role": "system", "content": SystemRole},
              {"role": "user", "content": rows[i]['reason']}
              ]
        )
        GPT4Tokens += len(encoding.encode(SystemRole + rows[i]['reason']))
        predicted_category = completion.choices[0].message.content
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        FT_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        print(str(i))
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
print(GPT4Tokens)

In [7]:
def extract_number(s):
    # Splitting the string into number and text
    parts = s.split('\n', 1)  # The '1' ensures the split occurs at the first occurrence of '\n'

    # Extracting the number and text
    number = parts[0]  # Convert the first part to an integer
    text = parts[1]         # The second part is the text
    
    return number

In [ ]:
# Test the gpt4 turbo on the validation set with the COT System Role.

FT_Category = []
Diff = []
count = 0
newCount = 0
savePredictionGPT4COT = []
GPT4cotTokens=0

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
    try:
        completion = client.chat.completions.create(
              model="gpt-4-turbo-preview",
              messages=[
              {"role": "system", "content": SystemRoleCOT},
              {"role": "user", "content": rows[i]['reason']}
              ]
        )
        GPT4cotTokens += len(encoding.encode(SystemRoleCOT + rows[i]['reason']))
        predicted_category = extract_number(completion.choices[0].message.content)
        savePredictionGPT4COT.append(int(predicted_category))
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        FT_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
print(GPT4cotTokens)

In [ ]:
# Test the gpt4 turbo on the validation set with including a bunch of prompt pairs.

import copy
import json

FT_Category = []
Diff = []
count = 0
newCount = 0
savePredictionGPT800FS0 = []

# Existing structure
message.clear()
message = [
    {"role": "system", "content": SystemRoleM}  # Assuming 'SystemRole' is a placeholder for actual content
]

# Loop over each entry in 'rows'

GPT4FS800Tokens = 0

for i in range(0,799):
    message.append({"role": "user", "content": rows[i]['reason']})
    message.append({"role": "assistant", "content": rows[i]['roy_categorized']})

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):

    NewMessage = copy.deepcopy(message)
    NewMessage.append({"role": "user", "content": rows[i]['reason']})
    try:
        
        completion = client.chat.completions.create(
              model="gpt-4-turbo-preview",
              messages=NewMessage
        )
        # Convert the entire list of dictionaries (messages) to a JSON string
        message_json = json.dumps(NewMessage)  # `indent=4` for pretty printing, optional

        GPT4FS800Tokens += len(encoding.encode(message_json))
        predicted_category = extract_number(completion.choices[0].message.content)
        predicted_category = completion.choices[0].message.content
        savePredictionGPT4FS0.append(int(predicted_category))
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        FT_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
print(GPT4FS800Tokens)



In [ ]:
# Test the gpt3.5 on the validation set with the System Role.

FT_Category = []
Diff = []
count = 0
newCount = 0
savePredictionGPT35 = []
GPT35Tokens = 0

for i in range(numOfTrainingSamples,numOfTrainingSamples+numOfTestingSamples):
    try:
        completion = client.chat.completions.create(
              model="gpt-3.5-turbo-0125",
              messages=[
              {"role": "system", "content": SystemRole},
              {"role": "user", "content": rows[i]['reason']}
              ]
        )
        predicted_category = completion.choices[0].message.content
        GPT35Tokens += len(encoding.encode(SystemRole + rows[i]['reason']))
        savePredictionGPT35.append(int(predicted_category))
        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        FT_Category.append(predicted_category)
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        count += 1
    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(newCount)
print(GPT35Tokens)

In [66]:
import pandas as pd
import os

file_path = 'AI_Validation.csv'

# Prepare DataFrame headers if starting a new file
if not os.path.exists(file_path):
    # Assuming you know the columns names in advance or construct them dynamically
    df = pd.DataFrame(columns=['random_order', 'reason', 'translation', 'roy_categorized', 
                               'GPT3.5 FineTune', 'GPT4 FS 800', 'GPT4 FS 400', 
                               'GPT4 FS 200', 'GPT4 FS 50', 'GPT4 FS 2', 'GPT4 Zero Shot',
                               'GPT3.5 Few Shot 2', 'GPT4 CoT'])
    df.to_csv(file_path, index=False)  # Write empty df with headers

# Append data to CSV file
with open(file_path, 'a', newline='') as f:
    count = 0
    for i in range(numOfTrainingSamples, numOfTrainingSamples + numOfTestingSamples):
        data = {
            'random_order': rows[i]['random_order'], 'reason': rows[i]['reason'],
            'translation': rows[i]['translation'], 'roy_categorized': rows[i]['roy_categorized'],
            'GPT3.5 FineTune': savePredictionFT[count], 'GPT4 FS 800': savePredictionGPT4FS800[count],
            'GPT4 FS 400': savePredictionGPT4FS400[count], 'GPT4 FS 200': savePredictionGPT4FS200[count],
            'GPT4 FS 50': savePredictionGPT4FS50[count], 'GPT4 FS 2': savePredictionGPT4FS2[count],
            'GPT4 Zero Shot': savePredictionGPT4FS0[count], 'GPT3.5 Few Shot 2': savePredictionGPT35[count],
            'GPT4 CoT': savePredictionGPT4COT[count]
        }
        count += 1
        df = pd.DataFrame([data])
        df.to_csv(f, header=False, index=False)


In [8]:
import pandas as pd

# Assuming `FT_Category` contains categorical data and `savePredictionFT` contains numerical predictions
random_order = [row['random_order'] for row in rows]

df = pd.DataFrame({
    'random_order': random_order,
    'FT_Category': FT_Category
})

# Save the DataFrame to a CSV file
df.to_csv('FT_Predictions.csv', index=False)

In [ ]:
# Test the gpt4o  on the validation set with the System Role and 800 examples

import copy
import json

GPT4o800FT_Category = []
Diff = []
count = 0
newCount = 0
savePredictionGPT800set = []
# Existing structure
#message.clear()
message = [
    {"role": "system", "content": SystemRoleM}  # Assuming 'SystemRole' is a placeholder for actual content
]

# Loop over each entry in 'rows'

GPT4oFS800Tokens = 0

for i in range(0,800):
    message.append({"role": "user", "content": rows[i]['reason']})
    message.append({"role": "assistant", "content": rows[i]['roy_categorized']})

for i in range(800,1000):

    NewMessage = copy.deepcopy(message)
    NewMessage.append({"role": "user", "content": rows[i]['reason']})
    try:
        
        completion = client.chat.completions.create(
              model="gpt-4o-2024-05-13",
              messages=NewMessage
        )
        # Convert the entire list of dictionaries (messages) to a JSON string
        message_json = json.dumps(NewMessage)  # `indent=4` for pretty printing, optional

        GPT4oFS800Tokens += len(encoding.encode(message_json))
        
        predicted_category = completion.choices[0].message.content
        GPT4o800FT_Category.append(predicted_category)
        savePredictionGPT800set.append(int(predicted_category))

        matching_numbers = find_matching_numbers(predicted_category,rows[i]['roy_categorized'])
        
        if matching_numbers:
            correct = True
        else: 
            correct = False
            
        Diff.append(correct)
        #count += 1

    except Exception as e:
        print("An error occurred:", str(e))
        print(rows[i]['roy_categorized'])
        GPT4o800FT_Category.append(0)
        Diff.append(False)
        newCount += 1

print(sum(Diff))
print(GPT4oFS800Tokens)